In [2]:
import os
import sys
import base64
from email.mime.text import MIMEText
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from gmail_auth import authenticate_gmail

In [3]:
service = authenticate_gmail()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=660506888931-38v48l8og4ea9dmlsis3jomva3751ujo.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A49510%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify&state=oSxyILTRFB93ECJdtRWZkW7NWHQnu1&access_type=offline


In [41]:
results = service.users().messages().list(
            userId="me", 
            maxResults=3,
            labelIds=["INBOX", "UNREAD"],        
            q="category:primary"
        ).execute()

In [42]:
results

{'messages': [{'id': '19607106b2b61275', 'threadId': '19607106b2b61275'},
  {'id': '196070cdb3e1450d', 'threadId': '196070cdb3e1450d'},
  {'id': '195ffef9f7f7c71a', 'threadId': '195ffef9f7f7c71a'}],
 'nextPageToken': '17380258890642508208',
 'resultSizeEstimate': 201}

In [43]:
messages = results.get("messages")
messages

[{'id': '19607106b2b61275', 'threadId': '19607106b2b61275'},
 {'id': '196070cdb3e1450d', 'threadId': '196070cdb3e1450d'},
 {'id': '195ffef9f7f7c71a', 'threadId': '195ffef9f7f7c71a'}]

In [69]:
msg = messages[0]
msg_data = service.users().messages().get(userId="me", id=msg["id"]).execute()
msg_data

{'id': '19607106b2b61275',
 'threadId': '19607106b2b61275',
 'labelIds': ['CATEGORY_UPDATES'],
 'snippet': 'Email not displaying correctly? View it online Email not displaying correctly? View it online Hi there, Thanks for joining the Google Developer Program. It&#39;s great to have you on board. The Google',
 'payload': {'partId': '',
  'mimeType': 'multipart/alternative',
  'filename': '',
  'headers': [{'name': 'Delivered-To', 'value': 'zhaowj1107@gmail.com'},
   {'name': 'Received',
    'value': 'by 2002:a8a:341:0:b0:590:96bf:dc35 with SMTP id z1csp5787567oct;        Sat, 5 Apr 2025 10:47:18 -0700 (PDT)'},
   {'name': 'X-Received',
    'value': 'by 2002:a05:622a:13:b0:478:f00f:4440 with SMTP id d75a77b69052e-47925a4096emr96843481cf.40.1743875238611;        Sat, 05 Apr 2025 10:47:18 -0700 (PDT)'},
   {'name': 'ARC-Seal',
    'value': 'i=1; a=rsa-sha256; t=1743875238; cv=none;        d=google.com; s=arc-20240605;        b=a2y7CoajDjLnfaZIWhvdSvkJ4lJo8vyyTpX2+0KHHh1BEyc1Y5bEm+KIGMZpXj

In [50]:
snippet = msg_data["snippet"]
snippet

'Email not displaying correctly? View it online Email not displaying correctly? View it online Hi there, Thanks for joining the Google Developer Program. It&#39;s great to have you on board. The Google'

In [58]:
headers = msg_data["payload"]["headers"]

In [59]:
sender = next((h["value"] for h in headers if h["name"].lower() == "from"), "Unknown Sender")

In [62]:
subject = next((h["value"] for h in headers if h["name"].lower() == "subject"), "No Subject")

In [63]:
subject

'Welcome to the Google Developer Program'

In [67]:
body = ""
if "parts" in msg_data["payload"]:
    for part in msg_data["payload"]["parts"]:
        if part["mimeType"] == "text/plain":
            if "data" in part["body"]:
                body = base64.urlsafe_b64decode(part["body"]["data"]).decode("utf-8")
                break
elif "body" in msg_data["payload"] and "data" in msg_data["payload"]["body"]:
    body = base64.urlsafe_b64decode(msg_data["payload"]["body"]["data"]).decode("utf-8")

In [68]:
body

"Email not displaying correctly? View it online  \r\n<https://notifications.google.com/g/vib/ANiao5qxxd2dMhWiS1loJ5J0niV8v_XaFWFbCxLHrATsuVZjTTOP5-OOs8I-UQdI5RnIKn5ZGCWXvTUTynHUM0Q19eFocl4uCoUGRcAEmnUgyyCvJR-ACdMyJxVZw-mg31Al5bI>\r\n\r\nWelcome to your Google Developer Profile\r\n\r\n<https://notifications.google.com/g/p/ANiao5oObfn11fJL7E_OKIGTRkTMmvv5zL6SsgKtKi8ZKyibzlDJGhRi6mvlAbqAMj926CQS6Q9ENL6O4MSPj1yX5ld9sNlB5pAabfUrGtMdhnNdC8SyLL_jIrAuqZ3XdEmfCTW5FlHvOCAu9Crz3K4UaMyXZh4NwBc6MnJDsK9zibnV3YUG5P5yptKwLMw>\r\n\r\nHi there,\r\n\r\nThanks for joining the Google Developer Program. It’s great to have you on  \r\nboard.\r\n\r\nThe Google Developer Program is designed to help you get the most out of  \r\nGoogle for Developers and accelerate your coding journey by making  \r\ndocumentation more useful, training more accessible, and achievements more  \r\ntangible. This program gives you access to new benefits such as AI-powered  \r\ntools using Gemini models, personalized experiences base